In [157]:
import os
import pandas as pd

# Salary NI

In [159]:
class salary_tools:
    
    def __init__(self,salary,income_tax_code):
        
        self.salary = salary
        self.income_tax_code = income_tax_code
    
    def ni(self):

        bracket = [0,9568,50270]
        percentage = [0,0.12,0.02]

        bracket.append(int(self.salary))
        bracket = [num for num in bracket if num <= self.salary]
        bracket = list(set(bracket)) 
        bracket.sort()

        marginals = []
        i = 0
        while i < len(bracket):
            if i > 0:
                b = (bracket[i]-bracket[i-1]) * percentage[i-1]
                marginals.append(b)
            i = i + 1
            
        ni = sum(marginals)
        
        return(ni)
    
    def tax(self):
        
        sal = self.salary
        allowance = self.income_tax_code * 10
        allowance_change = 50270
        
        bracket = [0,allowance,allowance_change,150000]
        percentage = [0,0.2,0.4,0.45]
        
        if sal >= 125140:
            allowance_change = allowance_change - allowance
            allowance = 0
            del bracket[0]
            del percentage[0]
        
        elif sal > 100000:
            a_ch = list(range(100000,125140,2))
            a_ch.append(sal)
            a_ch.sort()
            a_ch = [num for num in a_ch if num <= sal]
            a_ch = [num for num in a_ch if isinstance(num,int)]
            a_ch = [num for num in a_ch if num % 2 == 0]
            a_ch = list(set(a_ch))
            for i in range(len(a_ch)):
                a_ch[i] -= 100000
            del a_ch[0]            
            allowance = allowance - len(a_ch)
            allowance_change = allowance_change - len(a_ch)
            
        bracket = [0,allowance,allowance_change,150000]
        bracket.append(int(sal))
        bracket = [num for num in bracket if num <= sal]
        bracket = list(set(bracket)) 
        bracket.sort()

        marginals = []
        i = 0
        while i < len(bracket):
            i
            if i > 0:
                b = (bracket[i]-bracket[i-1]) * percentage[i-1]
                marginals.append(b)
            i = i + 1
            
        tx = sum(marginals)
        
        return(tx)

# Corporate Deductions

In [160]:
class corporate_tools:
    
    def __init__(self,turnover,salary,expenses):
        
        self.turnover = turnover
        self.salary = salary
        self.expenses = expenses

    def corp_ni(self):
        
        cni_pc = 0.138
        
        cni = self.salary * cni_pc
        
        return(cni)
    
    def total_exp(self):
        
        exp = self.salary + self.expenses + self.corp_ni()
        
        return(exp)
    
    def gross_profit(self):
        
        g_prof = self.turnover - self.total_exp()
        
        return(g_prof)
    
    def corp_tax(self):
        
        ctax_pc = 0.19
        
        if self.gross_profit() <= 0:
            ct = 0
        else:
            ct = self.gross_profit() * ctax_pc
        
        return(ct)
    
    def net_profit(self):
        
        n_prof = self.gross_profit() - self.corp_tax()
        
        return(n_prof)

    def dividend (self):
        
        if self.net_profit() <= 0:
            div = 0
        else:
            div = self.net_profit()
        
        return(div)

    def div_tax(self):

        bracket = [0,12570,50270,150000]
        percentage = [0,0.075,0.32,0.381]
        tax_free = 2000
        
        sal = self.salary
        take = sal + self.dividend() - tax_free
        
        if take <= 0:
            div_tx = 0
            
        else:
            bracket.append(int(take))
            bracket = [num for num in bracket if num <= take]
            bracket = list(set(bracket)) 
            bracket.sort()
            
            for i, n in enumerate(bracket):
                if n <= sal:
                    bracket[i] = sal

            lst = []
            i = 0
            while i < len(bracket):
                if i > 0:
                    b = (bracket[i]-bracket[i-1])* percentage[i-1]
                    lst.append(b)
                i = i + 1
                
            div_tx = sum(lst)
        
        return(div_tx)
    
    def div_take(self):

        d_take = self.dividend() - self.div_tax()

        return(d_take)

In [161]:
class contractor_tax:
    
    def __init__(self,turnover,salary,expenses,tax_code):
        
        self.turnover = turnover
        self.salary = salary
        self.expenses = expenses
        self.tax_code = tax_code
        
    def calculate(self):

        ni = salary_tools(self.salary,self.tax_code).ni()
        tax = salary_tools(self.salary,self.tax_code).tax()
        sal_take = self.salary - ni - tax

        cni = corporate_tools(self.turnover,self.salary,self.expenses).corp_ni()
        ctax = corporate_tools(self.turnover,self.salary,self.expenses).corp_tax()
        
        dividend = corporate_tools(self.turnover,self.salary,self.expenses).dividend()
        dtax = corporate_tools(self.turnover,self.salary,self.expenses).div_tax()
        d_take = corporate_tools(self.turnover,self.salary,self.expenses).div_take()
        
        state_theft = ni + tax + cni + ctax + dtax
        
        takehome = sal_take + d_take
        
        percentage_take = takehome/self.turnover
        
        output = [self.turnover,
                  self.expenses,
                  self.salary,
                  ni,
                  tax,
                  sal_take,
                  cni,
                  ctax,
                  dividend,
                  dtax,
                  d_take,
                  state_theft,
                  takehome,
                  percentage_take
                 ]

        return(output)

In [162]:
class income_tax:
    
    def __init__(self,salary,tax_code):
        
        self.salary = salary
        self.tax_code = tax_code
        
    def calculate(self):
        ni = salary_tools(self.salary,self.tax_code).ni()
        tax = salary_tools(self.salary,self.tax_code).tax()
        cni = corporate_tools(self.salary,self.tax_code).corp_ni()
        
        visible_theft = ni + tax
        takehome = self.salary - visible_theft
        
        perceived_take_pct = takehome/self.salary
        actual_take_pct = takehome/(self.salary+cni)
        
        output = [self.salary,
                  ni,
                  tax,
                  cni,
                  visible_theft,
                  takehome,
                  perceived_take_pct,
                  actual_take_pct]
        
        return(output)

# Input Values: Turnover and Expenses

In [158]:
#Gross Income
turnover = 70000
#Total Expenses (Not Including Salary)
expenses = 5000

# ^^^^^^"Click Cell", then Click "Run All"^^^^^^

In [163]:
i = 0
data = []

while i <= (turnover - expenses):
    x = contractor_tax(turnover,i,expenses,1257).calculate()
        
    data.append(x)
        
    i = i + 1
    
deds = pd.DataFrame(data, columns = ['Turnover',
                                     'Expenses',
                                     'Salary',
                                     'NI',
                                     'Tax',
                                     'Salary TAKEHOME',
                                     'Corporate NI',
                                     'Corporation Tax',
                                     'Dividend',
                                     'Dividend Tax',
                                     'Dividend TAKEHOME',
                                     'Total Theft',
                                     'Takehome',
                                     'Take(%)'
                                    ]
                 )
deds

,Turnover,Expenses,Salary,NI,Tax,Salary TAKEHOME,Corporate NI,Corporation Tax,Dividend,Dividend Tax,Dividend TAKEHOME,Total Theft,Takehome,Take(%)
0,70000,5000,0,0.00,0.0,0.00,0.000,12350.00000,52650.00000,2949.1,49700.90000,15299.10000,49700.90000,0.710013
1,70000,5000,1,0.00,0.0,1.00,0.138,12349.78378,52649.07822,2949.1,49699.97822,15299.02178,49700.97822,0.710014
2,70000,5000,2,0.00,0.0,2.00,0.276,12349.56756,52648.15644,2949.1,49699.05644,15298.94356,49701.05644,0.710015
3,70000,5000,3,0.00,0.0,3.00,0.414,12349.35134,52647.23466,2949.1,49698.13466,15298.86534,49701.13466,0.710016
4,70000,5000,4,0.00,0.0,4.00,0.552,12349.13512,52646.31288,2949.1,49697.21288,15298.78712,49701.21288,0.710017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,70000,5000,64996,5178.76,13430.4,46386.84,8969.448,0.00000,0.00000,0.0,0.00000,27578.60800,46386.84000,0.662669
64997,70000,5000,64997,5178.78,13430.8,46387.42,8969.586,0.00000,0.00000,0.0,0.00000,27579.16600,46387.42000,0.662677
64998,70000,5000,64998,5178.80,13431.2,46388.00,8969.724,0.00000,0.00000,0.0,0.00000,27579.72400,46388.00000,0.662686
64999,70000,5000,64999,5178.82,13431.6,46388.58,8969.862,0.00000,0.00000,0.0,0.00000,27580.28200,46388.58000,0.662694


In [165]:
deds['Error Check'] = deds['Total Theft'] + deds['Takehome']
deds

,Turnover,Expenses,Salary,NI,Tax,Salary TAKEHOME,Corporate NI,Corporation Tax,Dividend,Dividend Tax,Dividend TAKEHOME,Total Theft,Takehome,Take(%),Error Check
0,70000,5000,0,0.00,0.0,0.00,0.000,12350.00000,52650.00000,2949.1,49700.90000,15299.10000,49700.90000,0.710013,65000.000
1,70000,5000,1,0.00,0.0,1.00,0.138,12349.78378,52649.07822,2949.1,49699.97822,15299.02178,49700.97822,0.710014,65000.000
2,70000,5000,2,0.00,0.0,2.00,0.276,12349.56756,52648.15644,2949.1,49699.05644,15298.94356,49701.05644,0.710015,65000.000
3,70000,5000,3,0.00,0.0,3.00,0.414,12349.35134,52647.23466,2949.1,49698.13466,15298.86534,49701.13466,0.710016,65000.000
4,70000,5000,4,0.00,0.0,4.00,0.552,12349.13512,52646.31288,2949.1,49697.21288,15298.78712,49701.21288,0.710017,65000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,70000,5000,64996,5178.76,13430.4,46386.84,8969.448,0.00000,0.00000,0.0,0.00000,27578.60800,46386.84000,0.662669,73965.448
64997,70000,5000,64997,5178.78,13430.8,46387.42,8969.586,0.00000,0.00000,0.0,0.00000,27579.16600,46387.42000,0.662677,73966.586
64998,70000,5000,64998,5178.80,13431.2,46388.00,8969.724,0.00000,0.00000,0.0,0.00000,27579.72400,46388.00000,0.662686,73967.724
64999,70000,5000,64999,5178.82,13431.6,46388.58,8969.862,0.00000,0.00000,0.0,0.00000,27580.28200,46388.58000,0.662694,73968.862


In [166]:
deds['Error Check'] = deds['Total Theft'] + deds['Takehome']
deds

,Turnover,Expenses,Salary,NI,Tax,Salary TAKEHOME,Corporate NI,Corporation Tax,Dividend,Dividend Tax,Dividend TAKEHOME,Total Theft,Takehome,Take(%),Error Check
0,70000,5000,0,0.00,0.0,0.00,0.000,12350.00000,52650.00000,2949.1,49700.90000,15299.10000,49700.90000,0.710013,65000.000
1,70000,5000,1,0.00,0.0,1.00,0.138,12349.78378,52649.07822,2949.1,49699.97822,15299.02178,49700.97822,0.710014,65000.000
2,70000,5000,2,0.00,0.0,2.00,0.276,12349.56756,52648.15644,2949.1,49699.05644,15298.94356,49701.05644,0.710015,65000.000
3,70000,5000,3,0.00,0.0,3.00,0.414,12349.35134,52647.23466,2949.1,49698.13466,15298.86534,49701.13466,0.710016,65000.000
4,70000,5000,4,0.00,0.0,4.00,0.552,12349.13512,52646.31288,2949.1,49697.21288,15298.78712,49701.21288,0.710017,65000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64996,70000,5000,64996,5178.76,13430.4,46386.84,8969.448,0.00000,0.00000,0.0,0.00000,27578.60800,46386.84000,0.662669,73965.448
64997,70000,5000,64997,5178.78,13430.8,46387.42,8969.586,0.00000,0.00000,0.0,0.00000,27579.16600,46387.42000,0.662677,73966.586
64998,70000,5000,64998,5178.80,13431.2,46388.00,8969.724,0.00000,0.00000,0.0,0.00000,27579.72400,46388.00000,0.662686,73967.724
64999,70000,5000,64999,5178.82,13431.6,46388.58,8969.862,0.00000,0.00000,0.0,0.00000,27580.28200,46388.58000,0.662694,73968.862


# Best Scenario - This is your Maximum Takehome - PAY THIS - We are Minimising Rishi's State-Sponsored Theft

In [168]:
hi = deds[deds['Takehome'] == deds['Takehome'].max()]
hi

,Turnover,Expenses,Salary,NI,Tax,Salary TAKEHOME,Corporate NI,Corporation Tax,Dividend,Dividend Tax,Dividend TAKEHOME,Total Theft,Takehome,Take(%),Error Check,No Overspill
9568,70000,5000,9568,0.0,0.0,9568.0,1320.384,10281.20704,43830.40896,3188.46,40641.94896,14790.05104,50209.94896,0.717285,65000.0,True


In [169]:
gross_payout = hi['Salary'] + hi['Dividend']
gross_payout

9568    53398.40896
dtype: float64

In [170]:
turn_check = hi['Total Theft'] + hi['Takehome']
turn_check

9568    65000.0
dtype: float64